## Login to Low-side domain

In [ ]:
!pip install syft

In [ ]:
import syft as sy

In [ ]:
client = sy.login(url='http://20.206.188.97:8888', email='userxx@pet.com', password='userxx')
client

## Inspects the data assets

In [ ]:
import pandas as pd

In [ ]:
client.datasets

In [ ]:
client.datasets[0]

In [ ]:
client.datasets[0].assets[0]

In [ ]:
df = client.datasets[0].assets[0].mock
df

## Submitting a project and code requests to the low-side

In [ ]:
from opendp.measurements import make_base_laplace
from opendp.mod import enable_features
import opendp.prelude as dp

enable_features('contrib')

base_lap_vec_sugg = make_base_laplace(10., D='VectorDomain<AllDomain<float>>')

In [ ]:
tab = pd.crosstab(df['v0001_federation_unit'], df['v0614_eye_problems'])
tab

In [ ]:
tmp = []
for i in range(tab.shape[0]):
    tmp.append(base_lap_vec_sugg([float(x) for x in list(tab.iloc[i])]))

tab_dp = pd.DataFrame(tmp, index=tab.index, columns=tab.columns)
tab_dp

In [ ]:
def tab_1(data):
    import pandas as pd
    from opendp.measurements import make_base_laplace
    from opendp.mod import enable_features
    import opendp.prelude as dp

    enable_features("contrib")

    base_lap_vec_sugg = make_base_laplace(10., D="VectorDomain<AllDomain<float>>")

    tab = pd.crosstab(data['v0001_federation_unit'], data['v0614_eye_problems'])

    tmp = []
    for i in range(tab.shape[0]):
        tmp.append(base_lap_vec_sugg([float(x) for x in list(tab.iloc[i])]))

    tab_dp = pd.DataFrame(tmp, index=tab.index, columns=tab.columns)

    from io import BytesIO
    tabfile = BytesIO()
    tab_dp.to_pickle(tabfile)

    return tab_dp

tab_1(data=df)

### Creating Projects

In [ ]:
project_obj = sy.Project(
    name="Seeing difficulties",
    description="I would like to study how individuals in Brazil are affected by seeing difficulties",
    members=[client]
)

project = project_obj.start()

In [ ]:
project

In [ ]:
client.projects

### Requests within projects

In [ ]:
@sy.syft_function_single_use(data=client.datasets[0].assets[0])
def tab_1(data):
    import pandas as pd
    from opendp.measurements import make_base_laplace
    from opendp.mod import enable_features
    import opendp.prelude as dp

    enable_features("contrib")

    base_lap_vec_sugg = make_base_laplace(10., D="VectorDomain<AllDomain<float>>")

    tab = pd.crosstab(data['v0001_federation_unit'], data['v0614_eye_problems'])

    tmp = []
    for i in range(tab.shape[0]):
        tmp.append(base_lap_vec_sugg([float(x) for x in list(tab.iloc[i])]))

    tab_dp = pd.DataFrame(tmp, index=tab.index, columns=tab.columns)

    from io import BytesIO
    tabfile = BytesIO()
    tab_dp.to_pickle(tabfile)

    return tab_dp

tab_1(data=df)

## Send a code request

In [ ]:
project.create_code_request(tab_1, client)

In [ ]:
project.requests

In [ ]:
project.requests[0]

In [ ]:
project.requests[0].code

## Get result from a code request

In [ ]:
client.code.tab_1(data=client.datasets[0].assets[0])